In [9]:
import pandas as pd
import numpy as np

In [10]:
course = pd.read_excel("Course(Subjects).xlsx")
faculty = pd.read_excel("Faculty.xlsx")
room = pd.read_excel("Room.xlsx")

In [11]:
dfCourse = pd.DataFrame(course)
dfFaculty = pd.DataFrame(faculty)
dfRoom = pd.DataFrame(room)

dfCourse["divisions"] = dfCourse["divisions"].apply(
    lambda x: len(str(x).split(",")) if isinstance(x, str) else int(x)
)


num_cols = ["year", "divisions", "credit", "hours_per_week", "theory_hours", "practical_hours"]
dfCourse[num_cols] = dfCourse[num_cols].astype(int)

In [12]:
# we have lowercased the whole df
dfCourse = dfCourse.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))
dfFaculty = dfFaculty.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))
dfRoom = dfRoom.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))

In [14]:
# --- Constants ---
COLLEGE_START = 10*60 + 15   # 10:15 AM
COLLEGE_END   = 17*60 + 30   # 5:30 PM
THEORY_DURATION = 60         # 1 hour
WORKING_DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

BREAKS = [
    {"name": "Lunch Break", "start": 12*60 + 15, "duration": 60}, # 12:15–13:15
    {"name": "Tea Break", "start": 15*60 + 15, "duration": 15},   # 3:15–3:30
]

# --- Helper: Convert minutes → "HH:MM" string ---
def minutes_to_time(m):
    return f"{m//60:02d}:{m%60:02d}"

# --- Generate lecture + break slots ---
def generate_slots():
    slots = []
    t = COLLEGE_START
    while t < COLLEGE_END:
        brk = next((br for br in BREAKS if br["start"] == t), None)
        if brk:
            # keep only time, no "(Break Name)" in headers
            br_start = minutes_to_time(brk["start"])
            br_end = minutes_to_time(brk["start"] + brk["duration"])
            slots.append(f"{br_start}-{br_end}")
            t += brk["duration"]
        else:
            start = minutes_to_time(t)
            end = minutes_to_time(t + THEORY_DURATION)
            slots.append(f"{start}-{end}")
            t += THEORY_DURATION
    return slots

# --- Main Function: Create blank timetable for all years/divisions ---
def create_all_timetables(dfCourse):
    timetables = {}
    slots = generate_slots()

    # Group by year
    grouped = dfCourse.groupby("year")["divisions"].max()

    for year, max_divs in grouped.items():
        for div in range(1, max_divs+1):
            key = f"Year{year}_Div{div}"
            df = pd.DataFrame(
                [["" for _ in slots] for _ in WORKING_DAYS],
                index=WORKING_DAYS,
                columns=slots
            )
            timetables[key] = df
    return timetables


# --- Example Usage ---
# Dummy dfCourse
# dfCourse = pd.DataFrame({
#     "course_id": ["CS201", "CS202", "CS301", "CS401"],
#     "name": ["DSA", "OS", "AI", "ML"],
#     "program": ["CSE", "CSE", "CSE", "CSE"],
#     "year": [2, 2, 3, 4],
#     "divisions": [4, 4, 3, 2],
#     "credit": [4, 3, 3, 4],
#     "hours_per_week": [4, 3, 3, 4],
#     "theory_hours": [3, 2, 2, 3],
#     "practical_hours": [1, 1, 1, 1],
#     "lab_required": [True, True, False, True],
#     "central": [False, False, False, False]
# })

# Generate blank timetables
timetables = create_all_timetables(dfCourse)
print(timetables)

# Print one timetable skeleton with borders
# print("\nBlank Timetable (Year 2, Div 1):")
# print(timetables["Year2_Div1"].to_string(justify="center"))


{'Year2_Div1':     10:15-11:15 11:15-12:15 12:15-13:15 13:15-14:15 14:15-15:15 15:15-15:30  \
Mon                                                                           
Tue                                                                           
Wed                                                                           
Thu                                                                           
Fri                                                                           
Sat                                                                           

    15:30-16:30 16:30-17:30  
Mon                          
Tue                          
Wed                          
Thu                          
Fri                          
Sat                          , 'Year2_Div2':     10:15-11:15 11:15-12:15 12:15-13:15 13:15-14:15 14:15-15:15 15:15-15:30  \
Mon                                                                           
Tue                                             